In [37]:
## PATH settings
import os
import sys
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
sys.path.append(project_root)
COMMON_PATH = os.path.join(project_root, 'common')

In [38]:
## Bespoke class imports
from common.database.adatabase import ADatabase
from common.processor.processor import Processor as p
from common.extractor.alpaca_extractor import AlpacaExtractor
from financial_common.indicator.indicator import Indicator
from financial_common.risk.benchmark import Benchmark
from financial_common.portfolio_management.kpi import KPI
from financial_common.metric.metric import Metric
from financial_common.portfolio_management.portfolio import Portfolio
import numpy as np
import matplotlib.pyplot as plt

In [39]:
## Import standard packages
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from time import sleep
import copy

In [40]:
orivault = ADatabase("ori")
alp = AlpacaExtractor(paper=False)

In [41]:
orivault.connect()
portfolio_dictionary = orivault.retrieve("results").to_dict("records")[0]
orivault.disconnect()
portfolio_dictionary

{'ranking_metric': 'roc',
 'timeframe': 'DAY',
 'position_type': 'long',
 'grouping_type': 'STANDARD_DEV',
 'selection_type': 'bottom_blacklist',
 'allocation_type': 'risk',
 'risk_type': 'coefficient_of_variation',
 'selection_percentage': 1,
 'stoploss': 1,
 'num_of_groups': 1,
 'max_price': 1.0,
 'min_price': 0.1,
 'max_market_cap': 1000000,
 'min_market_cap': 10000,
 'rolling_window': 10,
 'leverage': 1,
 'date': Timestamp('2025-03-14 00:00:00'),
 'pnl': 1.0399155845575527,
 'raw_pnl': 1.074481883477311,
 'downside': 1.003024196303915,
 'coefficient_of_variation': 74.19476962085422,
 'portfolio_std': 0.014016022825755354,
 'sharpe_ratio': 2.7798101461288307,
 'tracking_error': 0.022069624622821006,
 'information_ratio': 1.7654075737708579}

In [42]:
pm = Portfolio.from_dict(portfolio_dictionary)
pm.to_dict()

{'ranking_metric': 'roc',
 'timeframe': 'DAY',
 'position_type': 'long',
 'grouping_type': 'STANDARD_DEV',
 'selection_type': 'bottom_blacklist',
 'allocation_type': 'risk',
 'risk_type': 'coefficient_of_variation',
 'selection_percentage': 1.0,
 'stoploss': 1.0,
 'num_of_groups': 1,
 'max_price': 1.0,
 'min_price': 0.1,
 'max_market_cap': 1000000.0,
 'min_market_cap': 10000.0,
 'rolling_window': 10,
 'leverage': 1}

In [43]:
## Retrieve existing stocks 
index = alp.assets().sort_values("ticker")
tickers = index["ticker"].values
tickers_per_batch = int(1000)
batchs = [tickers[i:i + tickers_per_batch] for i in range(0, len(tickers), tickers_per_batch)]
bars = []
[bars.append(alp.latest_bars_bulk(batch)) for batch in batchs]
todays_bar = pd.concat(bars)
index = index.merge(todays_bar,on="ticker",how="left")

'SVA'
'WTGUU'


In [44]:
end = alp.clock() - timedelta(days=1)
start = (end - timedelta(days=pm.rolling_window*3))
print(start,end)

2025-04-26 14:45:51.010543-04:00 2025-05-26 14:45:51.010543-04:00


In [45]:
tickers_per_batch = int(10000/(pm.rolling_window*4)/5)
print(tickers_per_batch)

50


In [46]:
relevant_tickers = index[(index["adjclose"]>=pm.min_price) & (index["adjclose"]<=pm.max_price)]["ticker"]
batchs = [relevant_tickers[i:i + tickers_per_batch] for i in range(0, len(relevant_tickers), tickers_per_batch)]

In [47]:
prices = []
for batch in tqdm(batchs):
    try:
        tickers_data = alp.prices_bulk(batch,start,end)
        for ticker in batch:
            try:
                price = tickers_data[tickers_data["ticker"] == ticker].copy()
                price = p.lower_column(price)
                price = p.utc_date(price)
                price.sort_values("date", inplace=True)
                price = p.additional_date_columns(price)
                price["market_cap"] = price["adjclose"] * price["volume"]
                if price[(price["adjclose"]<=pm.max_price) & (price["adjclose"]>=pm.min_price)].index.size > 0:
                    price["standard_dev"] = 1
                    price["coefficient_of_variation"] = 1
                    price = Indicator.indicator_type_factory(pm.ranking_metric).calculate(price,timeframe=pm.rolling_window,live=True)
                    prices.append(price)
            except Exception as e:
                print(str(e))
    except Exception as e:
        print(str(e))

 85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                                | 11/13 [00:07<00:01,  1.35it/s]

'UYSCR'


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 13/13 [00:09<00:00,  1.42it/s]


In [48]:
simulation = pd.concat(prices)
simulation.sort_values("date", inplace=True)
trades = pm.recs(simulation.copy())
trades

,year,day,ticker,date,adjclose,high,low,volume,month,quarter,...,adjclose_test,high_test,low_test,volume_test,roc,group_percentile,rank_percentile,major_key,position_type,weight
19,2025,143,CHACR,2025-05-23 00:00:00+00:00,0.21,0.21,0.21,500000,5,2,...,0.2,0.2145,0.2,1144200.0,NaN,1,1000,2025_143_1,1.0,1.0
